In [1]:
import hid
import time

# Konstanty pro VID a PID
VID = 0x0403  # Nahraďte skutečným VID vašeho zařízení
PID = 0x6030  # Nahraďte skutečným PID vašeho zařízení
INTERFACE = 1

In [2]:
# Tady se nalezne spravny HID zarizeni dle VID/PID a vybere se UART interface

devices = hid.enumerate()

for d in devices:
    if d['vendor_id'] == VID and d['product_id'] == PID:
        print("Nalezeno zařízení:", d)

        if d['interface_number'] == INTERFACE:
            try:
                device = hid.device()
                device.open_path(d['path'])
            finally:
                print("Vybrane zarizeni", device)
                device.close()

Nalezeno zařízení: {'path': b'5-2.2:1.0', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 0}
Nalezeno zařízení: {'path': b'5-2.2:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}
Vybrane zarizeni <hid.device object at 0x7faed1103d80>


In [3]:
# Otevrit HID zarizeni

device = hid.device()
device.open_path(d['path'])

{'path': b'5-2.2:1.1', 'vendor_id': 1027, 'product_id': 24624, 'serial_number': '', 'release_number': 8704, 'manufacturer_string': 'FTDI', 'product_string': 'FT260', 'usage_page': 0, 'usage': 0, 'interface_number': 1}


In [50]:
device.close()

In [ ]:
# Vypis zakladni parametry o zarizeni 
try:
    print(f'Device manufacturer: {device.get_manufacturer_string()}')
    print(f'Product: {device.get_product_string()}')
    print(f'Serial Number: {device.get_serial_number_string()}')

    device.set_nonblocking(1)

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)


In [ ]:
# Smycka, ktera vycita RX buffer u FTDI

try:
    while True:
        # Čtení dat
        data = device.read(64)  # Přečte až 64 bajtů dat
        if data:
            print("Přijatá data:", [hex(x) for x in data] )
            device.write(data[2])

        time.sleep(0.05)  # Krátká pauza pro snížení zatížení procesoru

except IOError as e:
    print("Chyba při práci s HID zařízením:", e)
#finally:
#    # Uzavření zařízení
#    #device.close()


Tohle je funkcni blok, ktery posle obsah promenne 'zprava' po seriove lince. 

In [ ]:
# Poslat UART zpravu do arduina

zprava = b'\n\r Ahoj Testovaci zprava'
l = len(zprava)
payload = bytes([0xFE, l]) + zprava
print(l, payload)
device.write(payload)

In [ ]:
device.read(100)

In [19]:
help(device.get_feature_report)

Help on built-in function get_feature_report:

get_feature_report(...) method of hid.device instance
    Receive feature report.
    
    :param report_num:
    :type report_num: int
    :param max_length:
    :type max_length: int
    :return: Incoming feature report
    :rtype: List[int]
    :raises ValueError: If connection is not opened.
    :raises IOError:



In [18]:
d = device.get_feature_report(0xA1, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)

print('flow_ctrl: {}'.format(d[2]))

25
['0xa1', '0x3', '0x2', '0x0', '0x1', '0x1', '0x3', '0x0', '0x1', '0x3', '0x6', '0x1', '0x1', '0xc', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0xac', '0x0', '0xbc', '0x0', '0xcc']
flow_ctrl: 2


In [5]:
# Vypsani aktualniho nastaveni TF260 cipu

d = device.get_feature_report(0xE0, 100)
da = [hex(x) for x in d]
print(len(d))
print(da)
print("")
print('flow_ctrl: {}'.format(d[1]))
print('baud_rade: {}'.format([hex(x) for x in d[2:6]]))
print('data_bit: {}'.format(d[6]))
print('parity: {}'.format(d[7]))
print('stop_bit: {}'.format(d[8]))

10
['0xe0', '0x3', '0x80', '0x25', '0x0', '0x0', '0x8', '0x0', '0x0', '0x0']

flow_ctrl: 3
baud_rade: ['0x80', '0x25', '0x0', '0x0']
data_bit: 8
parity: 0
stop_bit: 0


In [7]:
device.send_feature_report([0xA1, 0x41, 0x3, 0x00, 0x4B, 0x00, 0x00, 0x08, 0x01, 0x01, 0x00])

-1

In [ ]:
# Timto se nastavi baudrate na 9600

device.send_feature_report([0xA1, 0x42, 0x80, 0x25, 0x00, 0x00])

In [ ]:
# Vypnuti LEDEk (resp. prepnuti do RX/TX modu)

device.send_feature_report([0xA1, 0x08, 0x04])  # Nastavit GPIOA do TX_LED
device.send_feature_report([0xA1, 0x09, 0x05])  # Nastavit GPIOG do RX_LED